In [65]:
!gdown --id 1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU
From (redirected): https://drive.google.com/uc?id=1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU&confirm=t&uuid=ea273308-4cd3-4534-94b6-9bc38a34be56
To: /kaggle/working/nature_12K.zip
 29%|███████████▎                           | 1.11G/3.82G [00:04<00:10, 261MB/s]^C
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gdown/__main__.py", line 172, in main
    download(
  File "/usr/local/lib/python3.11/dist-packages/gdown/download.py", line 368, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/usr/local/lib/python3.11/dist-packages/requ

In [3]:
!unzip -q nature_12K.zip

In [ ]:
https://drive.google.com/file/d/1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU/view?usp=sharing

In [82]:
from torch.nn import Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d, LeakyReLU, GELU, SELU, Mish
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math
import matplotlib.pyplot as plt
import numpy as np

In [83]:
PARAMETERS = {
    'data_augmentation': True,
    'batch_normalization': False,
    'filters': 64, # no. of filters in first layer
    'filter_org': 'half', # 'half', 'double'
    'dropout': 0.2,
    'activation': 'relu',
    'train_data_dir': "/kaggle/working/inaturalist_12K/train",
    'test_data_dir': "/kaggle/working/inaturalist_12K/val",
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 25,
    'dim': 256,
    'conv_kernel_size': 3,
    'dense_neurons': 1000
}

In [84]:
CLASSES = {
    0:'Amphibia',
    1:'Animalia',
    2:'Arachnida',
    3:'Aves',
    4:'Fungi',
    5:'Insecta',
    6:'Mammalia',
    7:'Mollusca',
    8:'Plantae',
    9:'Reptilia'
}

In [85]:
def get_data(param, type):
    if(type.lower() == 'train'):
        if param['data_augmentation']:
            transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.3),
                transforms.RandomRotation(degrees=12),
                transforms.Resize((param['dim'],param['dim'])),
                transforms.ToTensor(), 
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize((param['dim'],param['dim'])),
                transforms.ToTensor(), 
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
            ])

        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    
    else:
        transform = transforms.Compose([
            transforms.Resize((param['dim'],param['dim'])),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        test_dataset = datasets.ImageFolder(root=param['test_data_dir'], transform=transform)
        test_dataloader = DataLoader(test_dataset, batch_size=param['batch_size'])
        return test_dataloader

    

In [87]:
class CNN(Module):
    def __init__(self, param):
        super(CNN, self).__init__()
        self.param=param
        self.data_augmentation = param['data_augmentation']
        self.dropout = param['dropout']
        self.act = self.getActivation(param['activation'])
        self.filters = self.filter_logic(param['filters'], param['filter_org'])
        self.conv_ks = param['conv_kernel_size']
        self.dim = param['dim']
        self.bn = param['batch_normalization']
        self.dense_neurons = param['dense_neurons']


        ####### Layer 1 #######
        curr_dim = self.dim
        self.conv1 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=3, out_channels=self.filters[0])
        curr_dim -= (self.conv_ks-1)
        self.act1 = self.act
        if(self.bn): self.bn1 = BatchNorm2d(self.filters[0])
        self.pool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout1 = Dropout(p=self.dropout)

        ####### Layer 2 #######
        self.conv2 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[0], out_channels=self.filters[1])
        curr_dim -= (self.conv_ks-1)
        self.act2 = self.act
        if(self.bn): self.bn2 = BatchNorm2d(self.filters[1])
        self.pool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout2 = Dropout(p=self.dropout)

        ####### Layer 3 #######
        self.conv3 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[1], out_channels=self.filters[2])
        curr_dim -= (self.conv_ks-1)
        self.act3 = self.act
        if(self.bn): self.bn3 = BatchNorm2d(self.filters[2])
        self.pool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout3 = Dropout(p=self.dropout)

        ####### Layer 4 #######
        self.conv4 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[2], out_channels=self.filters[3])
        curr_dim -= (self.conv_ks-1)
        self.act4 = self.act
        if(self.bn): self.bn4 = BatchNorm2d(self.filters[3])
        self.pool4 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout4 = Dropout(p=self.dropout)


        ####### Layer 5 #######
        self.conv5 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[3], out_channels=self.filters[4])
        curr_dim -= (self.conv_ks-1)
        self.act5 = self.act
        if(self.bn): self.bn5 = BatchNorm2d(self.filters[4])
        self.pool5 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout5 = Dropout(p=self.dropout)

    
        ####### Fully Connected Layer #######
        self.dense_neurons = curr_dim * curr_dim * self.filters[4]
        self.fc1 = Linear(in_features=(curr_dim * curr_dim * self.filters[4]), out_features=self.dense_neurons)  # How to calculate dimension of filters at previous level
        self.act6 = self.act
        self.dropout6 = Dropout(p=0.5)
        

        ####### Output Layer #######
        self.out = Linear(in_features=self.dense_neurons, out_features=10)
        self.act7 = LogSoftmax(dim=1)


    def getActivation(self, act):
        act = act.lower()
        if(act == 'relu'):
            return ReLU()
        elif(act == 'leakyrelu'):
            return LeakyReLU()
        elif(act == 'gelu'):
            return GELU()
        elif(act == 'selu'):
            return SELU()
        elif(act == 'mish'):
            return Mish()
    

    def filter_logic(self, filter, org):
        level = []
        org = org.lower()
        if org == 'same':
            level = [filter for i in range(5)]
        elif org == 'double':
            level = [filter*pow(2,i) for i in range(5)]
        elif org == 'half':
            level = [max(filter//pow(2,i),1) for i in range(5)]
        return level

    

    def forward(self, r):

        r=self.conv1(r)
        r=self.act1(r)
        if(self.bn): r=self.bn1(r)
        r=self.pool1(r)
        r=self.dropout1(r)

        r=self.conv2(r)
        r=self.act2(r)
        if(self.bn): r=self.bn2(r)
        r=self.pool2(r)
        r=self.dropout2(r)

        r=self.conv3(r)
        r=self.act3(r)
        if(self.bn): r=self.bn3(r)
        r=self.pool3(r)
        r=self.dropout3(r)

        r=self.conv4(r)
        r=self.act4(r)
        if(self.bn): r=self.bn4(r)
        r=self.pool4(r)
        r=self.dropout4(r)

        r=self.conv5(r)
        r=self.act5(r)
        if(self.bn): r=self.bn5(r)
        r=self.pool5(r)
        r=self.dropout5(r)

        r=flatten(r,1)
        r=self.fc1(r)
        r=self.act6(r)
        r=self.dropout6(r)
        
        r=self.out(r)
        output=self.act7(r)

        return output
        


In [15]:
# # For personal debug use
# def train(param):
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = CNN(param).to(device)
#     optimizer = Adam(model.parameters(), lr=param['learning_rate'])
#     loss_function = NLLLoss()
#     train_data_loader, validation_data_loader = get_data(param, 'train')
    

#     for epo in range(param['epochs']):
#         model.train()
#         totalTrainLoss = 0
#         totalValLoss = 0
#         trainCorrect = 0
#         valCorrect = 0
#         train_counter=0
#         validation_counter=0
#         for (image, label) in train_data_loader:
#             (image, label) = (image.to(device), label.to(device))
#             prediction = model(image)
#             loss = loss_function(prediction, label)
            
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             totalTrainLoss += loss
#             trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
#             train_counter+=1
#             # print(train_counter)
        
#         with no_grad():
#             model.eval()
#             for (image, label) in validation_data_loader:
#                 (image, label) = (image.to(device), label.to(device))
#                 pred = model(image)
#                 totalValLoss += loss_function(pred, label)
#                 valCorrect += (pred.argmax(1) == label).type(float).sum().item()
#                 validation_counter+=1

#         print(f"Epochs --> {epo}")
#         print(f"Train Loss --> {(totalTrainLoss/train_counter).cpu().detach().numpy()}")
#         print(f"Train Accuracy --> {trainCorrect/len(train_data_loader.dataset)}")
#         print(f"Validation Loss --> {(totalValLoss/validation_counter).cpu().detach().numpy()}")
#         print(f"Validation Accuracy --> {valCorrect/len(validation_data_loader.dataset)}")
#         print("##########################################################################################")
    
#     return model

In [16]:
mdl = train(PARAMETERS)

Epochs --> 0
Train Loss --> 2.278223991394043
Train Accuracy --> 0.1295
Validation Loss --> 2.2322638034820557
Validation Accuracy --> 0.16858429214607304
##########################################################################################
Epochs --> 1
Train Loss --> 2.1942501068115234
Train Accuracy --> 0.1955
Validation Loss --> 2.193485736846924
Validation Accuracy --> 0.21410705352676337
##########################################################################################
Epochs --> 2
Train Loss --> 2.1620495319366455
Train Accuracy --> 0.21425
Validation Loss --> 2.139416456222534
Validation Accuracy --> 0.21060530265132565
##########################################################################################
Epochs --> 3
Train Loss --> 2.129056453704834
Train Accuracy --> 0.228125
Validation Loss --> 2.1208577156066895
Validation Accuracy --> 0.23861930965482742
##########################################################################################
Epochs --> 4


KeyboardInterrupt: 

In [ ]:
# test_data_loader = get_data(PARAMETERS, 'test')
# tstCorrect = 0
# tstCounter = 0
# y = []
# y_pred = []
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# with no_grad():
#     mdl.eval()
#     for (image, label) in test_data_loader:
#         (image, label) = (image.to(device), label.to(device))
#         pred = mdl(image)
#         y.extend(label.tolist())
#         y_pred.extend(pred.argmax(1).tolist())
#         # print(pred)
#         tstCorrect += (pred.argmax(1) == label).type(float).sum().item()
#         tstCounter+=PARAMETERS['batch_size']

# print(tstCorrect)
# print(tstCounter)
# print((tstCorrect/tstCounter)*100)

In [88]:
## for wandb sweeps
def train():
    wandb.init()
    param = wandb.config
    wandb.run.name = f'fltr_{param.filters}_fltrOrg_{param.filter_org}_dataAug_{param.data_augmentation}_batchNorm_{param.batch_normalization}_act_{param.activation}_batchSz_{param.batch_size}'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = CNN(param).to(device)
    optimizer = Adam(model.parameters(), lr=param['learning_rate'])
    loss_function = NLLLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')
    

    for epo in range(param['epochs']):
        model.train()
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = model(image)
            loss = loss_function(prediction, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1
          
        
        with no_grad():
            model.eval()
            for (image, label) in validation_data_loader:
                (image, label) = (image.to(device), label.to(device))
                pred = model(image)
                totalValLoss += loss_function(pred, label)
                valCorrect += (pred.argmax(1) == label).type(float).sum().item()
                validation_counter+=1

        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)
        print(f"Epoch --> {epo}")
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        
        lg={
            'epoch': epo+1,
            'tr_accuracy': tr_acc,
            'val_accuracy': val_acc,
            'tr_loss': tr_ls,
            'val_loss': val_ls
        }
        wandb.log(lg)

        # return model

In [89]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [54]:
# sweep_config = {
#   "method": "bayes", 
#   "name": "Q2 Sweep",
#   "metric": {"goal": "maximize", "name": "val_accuracy"},
#   "parameters": {
#     "data_augmentation":{"values": [True,False]},  # List of boolean values
#     "batch_normalization":{"values": [True,False]},  # List of boolean values
#     "filters":{"values": [32]},  # List of filter values for first layer
#     "filter_org":{"values": ["same","half","double"]},  # List of filter organization options.. , "half", "double"
#     "dropout":{"values": [0.2,0.3]},  # Dropout rates
#     "activation":{"values": ["relu","leakyrelu","gelu","mish","selu"]},  # Activation functions... , "relu", "leaky_relu"
#     "batch_size":{"values": [32]},
#     "learning_rate":{"values": [0.001,0.0001]},
#     "epochs":{"values": [10]},
#     "dim":{"values": [256]},
#     "conv_kernel_size":{"values": [3,5]},
#     "dense_neurons":{"values": [512]},
#     "train_data_dir":{"values": ["/kaggle/working/inaturalist_12K/train"]},
#     "test_data_dir":{"values": ["/kaggle/working/inaturalist_12K/val"]}
#   }
# }


In [92]:
sweep_config = {
  "method": "bayes", 
  "name": "Q2 Sweep(modified run)",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "data_augmentation":{"values": [True]},  # List of boolean values
    "batch_normalization":{"values": [True]},  # List of boolean values
    "filters":{"values": [32,64]},  # List of filter values for first layer
    "filter_org":{"values": ["half"]},  # List of filter organization options.. , "half", "double"
    "dropout":{"values": [0.1,0.2]},  # Dropout rates
    "activation":{"values": ["relu"]},  # Activation functions... , "relu", "leaky_relu"
    "batch_size":{"values": [32,64]},
    "learning_rate":{"values": [0.0001,0.00001]},
    "epochs":{"values": [10,15]},
    "dim":{"values": [256]},
    "conv_kernel_size":{"values": [3]},
    "dense_neurons":{"values": [128,256]},
    "train_data_dir":{"values": ["/kaggle/working/inaturalist_12K/train"]},
    "test_data_dir":{"values": ["/kaggle/working/inaturalist_12K/val"]}
  }
}


In [76]:
wandb.login(key="b8d44a4abbab8753e976a6e5ab717fd669ba99a2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [93]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="DA6401_A2")

Create sweep with ID: 3yycib0m
Sweep URL: https://wandb.ai/cs24m030-indian-institute-of-technology-madras/DA6401_A2/sweeps/3yycib0m


In [ ]:
wandb.agent(sweep_id, function=train, count=20)
wandb.finish()

wandb: Agent Starting Run: s91pmoym with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 128
wandb: 	dim: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 10
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.3628880977630615
Train Accuracy --> 0.114125
Validation Loss --> 2.2633745670318604
Validation Accuracy --> 0.1570785392696348
Epoch --> 1
Train Loss --> 2.264404773712158
Train Accuracy --> 0.154875
Validation Loss --> 2.2116682529449463
Validation Accuracy --> 0.1805902951475738
Epoch --> 2
Train Loss --> 2.2123587131500244
Train Accuracy --> 0.180375
Validation Loss --> 2.1839001178741455
Validation Accuracy --> 0.21160580290145073
Epoch --> 3
Train Loss --> 2.1827433109283447
Train Accuracy --> 0.19475
Validation Loss --> 2.169598340988159
Validation Accuracy --> 0.22611305652826413
Epoch --> 4
Train Loss --> 2.157151699066162
Train Accuracy --> 0.214125
Validation Loss --> 2.151550769805908
Validation Accuracy --> 0.23261630815407705
Epoch --> 5
Train Loss --> 2.134291410446167
Train Accuracy --> 0.22075
Validation Loss --> 2.1425416469573975
Validation Accuracy --> 0.23611805902951477
Epoch --> 6
Train Loss --> 2.1183714866638184
Train Accuracy --> 0.

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▁▃▄▅▆▆▇▇██
tr_loss,█▆▄▄▃▃▂▂▁▁
val_accuracy,▁▃▆▇██▆▆▆█
val_loss,█▅▄▃▂▂▂▂▂▁
epoch,10
tr_accuracy,0.2515
tr_loss,2.06261
val_accuracy,0.23412
val_loss,2.12455


wandb: Agent Starting Run: y2s84utx with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 128
wandb: 	dim: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.3866569995880127
Train Accuracy --> 0.11375
Validation Loss --> 2.279421329498291
Validation Accuracy --> 0.14157078539269635
Epoch --> 1
Train Loss --> 2.3005077838897705
Train Accuracy --> 0.138875
Validation Loss --> 2.2413957118988037
Validation Accuracy --> 0.17508754377188596
Epoch --> 2
Train Loss --> 2.2526588439941406
Train Accuracy --> 0.164125
Validation Loss --> 2.2180275917053223
Validation Accuracy --> 0.1950975487743872
Epoch --> 3
Train Loss --> 2.2138218879699707
Train Accuracy --> 0.178875
Validation Loss --> 2.202569007873535
Validation Accuracy --> 0.1950975487743872
Epoch --> 4
Train Loss --> 2.1827263832092285
Train Accuracy --> 0.199
Validation Loss --> 2.189127206802368
Validation Accuracy --> 0.20060030015007504
Epoch --> 5
Train Loss --> 2.1658473014831543
Train Accuracy --> 0.2035
Validation Loss --> 2.184419631958008
Validation Accuracy --> 0.19709854927463732
Epoch --> 6
Train Loss --> 2.1390531063079834
Train Accuracy --> 0.214

...

wandb: Agent Starting Run: l2rfs78k with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 32
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 128
wandb: 	dim: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_org: half
wandb: 	filters: 32
wandb: 	learning_rate: 1e-05
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.5039823055267334
Train Accuracy --> 0.101125
Validation Loss --> 2.3416497707366943
Validation Accuracy --> 0.11105552776388194
Epoch --> 1
Train Loss --> 2.486469268798828
Train Accuracy --> 0.104625
Validation Loss --> 2.3350324630737305
Validation Accuracy --> 0.10855427713856929
Epoch --> 2
Train Loss --> 2.4731125831604004
Train Accuracy --> 0.103
Validation Loss --> 2.3296966552734375
Validation Accuracy --> 0.10355177588794397
Epoch --> 3
Train Loss --> 2.4542360305786133
Train Accuracy --> 0.107375
Validation Loss --> 2.32496976852417
Validation Accuracy --> 0.10055027513756878
Epoch --> 4
Train Loss --> 2.448817491531372
Train Accuracy --> 0.105625
Validation Loss --> 2.3203396797180176
Validation Accuracy --> 0.10055027513756878
Epoch --> 5
Train Loss --> 2.449557065963745
Train Accuracy --> 0.099
Validation Loss --> 2.31778621673584
Validation Accuracy --> 0.10455227613806903
Epoch --> 6
Train Loss --> 2.425358533859253
Train Accuracy --> 0.10412

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▃▆▄█▇▁▅▄▂▄
tr_loss,█▇▆▅▄▄▃▂▂▁
val_accuracy,▇▅▃▁▁▃▆█▇▇
val_loss,█▇▆▅▄▃▂▂▁▁
epoch,10
tr_accuracy,0.103
tr_loss,2.39579
val_accuracy,0.11256
val_loss,2.3089


wandb: Agent Starting Run: bbjyqh8t with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 32
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 256
wandb: 	dim: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 15
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.325293779373169
Train Accuracy --> 0.134875
Validation Loss --> 2.2333457469940186
Validation Accuracy --> 0.1775887943971986
Epoch --> 1
Train Loss --> 2.218285322189331
Train Accuracy --> 0.18875
Validation Loss --> 2.1622917652130127
Validation Accuracy --> 0.22611305652826413
Epoch --> 2
Train Loss --> 2.155552864074707
Train Accuracy --> 0.216875
Validation Loss --> 2.1471009254455566
Validation Accuracy --> 0.2221110555277639
Epoch --> 3
Train Loss --> 2.12343168258667
Train Accuracy --> 0.236875
Validation Loss --> 2.121326208114624
Validation Accuracy --> 0.24012006003001501
Epoch --> 4
Train Loss --> 2.1076955795288086
Train Accuracy --> 0.243625
Validation Loss --> 2.1007559299468994
Validation Accuracy --> 0.2546273136568284
Epoch --> 5
Train Loss --> 2.077974319458008
Train Accuracy --> 0.246375
Validation Loss --> 2.0790679454803467
Validation Accuracy --> 0.2656328164082041
Epoch --> 6
Train Loss --> 2.0553228855133057
Train Accuracy --> 0.257